# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [2]:
# Your code here:
salaries = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')
salaries.describe()

,Typical Hours,Annual Salary,Hourly Rate
count,8022.000000,25161.000000,8022.000000
mean,34.507604,86786.999790,32.788558
std,9.252077,21041.354602,12.112573
min,10.000000,7200.000000,2.650000
25%,20.000000,76266.000000,21.200000
50%,40.000000,90024.000000,35.600000
75%,40.000000,96060.000000,40.200000
max,40.000000,300000.000000,109.000000


Examine the `salaries` dataset using the `head` function below.

In [3]:
# Your code here:

salaries.head(50)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86
5,"ABBASI, CHRISTOPHER",STAFF ASST TO THE ALDERMAN,CITY COUNCIL,F,Salary,NaN,50436.0,NaN
6,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,F,Hourly,40.0,NaN,46.10
7,"ABBATE, JOSEPH L",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
8,"ABBATEMARCO, JAMES J",FIRE ENGINEER-EMT,FIRE,F,Salary,NaN,103350.0,NaN
9,"ABBATE, TERRY M",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
# Your code here:

# use isnull and the sum function in order to find missing data

salaries.isnull().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [5]:
# Your code here:

# by selecting the columns that we want to compare and use the isnull function we can count the entries 

salaries[['Hourly Rate','Annual Salary']].isnull().sum()


Hourly Rate      25161
Annual Salary     8022
dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
# Your code here:

# using the group by function 

emp_deparment = salaries.groupby(by='Department')['Name'].count()
emp_deparment.sort_values(ascending=False)

Department
POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [41]:
# Your code here:
#just select the employes that are paid on an hourly basis 

hourly_rates =  salaries['Hourly Rate'][salaries['Hourly Rate'].notnull()]

# Terms
hourly_mean = hourly_rates.mean()
mu = 30
sample_stdev = hourly_rates.std()
standard_error = sample_stdev / np.sqrt(len(hourly_rates))

from scipy.stats import t
from scipy import stats

t, p = stats.ttest_1samp(hourly_rates, 30)
print(t)
print(p)

20.6198057854942
4.3230240486229894e-92


In [57]:
hourly_rates

4        19.86
6        46.10
7        35.60
10        2.65
18       17.68
         ...  
33164    46.10
33168    17.68
33169    35.60
33174    46.35
33175    48.85
Name: Hourly Rate, Length: 8022, dtype: float64

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [62]:
# Your code here:

police_rates =  salaries['Annual Salary'][(salaries['Department']=='POLICE') & (salaries['Annual Salary'].notnull()) ]

alpha = 0.05

sample_mean = police_rates.mean()
mu = 86000
sample_stdev = police_rates.std()
standard_error = sample_stdev / np.sqrt(len(police_rates))

t_value = (sample_mean - mu) / standard_error
print(t_value)

t, p = stats.ttest_1samp(police_rates, mu)
print(t)
print(p)

3.08199700571315
3.081997005712994
0.0020603403550965137


Using the `crosstab` function, find the department that has the most hourly workers. 

In [44]:
# Your code here:

pd.crosstab(salaries.Department,values=salaries['Hourly Rate'],aggfunc='count',columns='Employees').sort_values(by='Employees',ascending=False)

col_0,Employees
Department,
STREETS & SAN,1862
WATER MGMNT,1513
OEMC,1273
AVIATION,1082
GENERAL SERVICES,765
TRANSPORTN,725
PUBLIC LIBRARY,299
FAMILY & SUPPORT,287
CITY COUNCIL,64


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [46]:
# Your code here:
from scipy.stats import ttest_1samp
street_san_sal=salaries['Hourly Rate'][salaries['Department']=='STREETS & SAN'].notnull()

tscore, pvalue = stats.ttest_1samp(street_san_sal, popmean=35)

print(tscore)
print(pvalue)

-4448.467241449082
0.0


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [70]:
# Your code here:

from scipy.stats import t
t.interval(0.95, len(hourly_rates)-1, loc=hourly_mean, scale=stats.sem(hourly_rates, axis=None, ddof=0))  # 95% confidence interval


(32.523474868671975, 33.05364056389059)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [71]:
# Your code here:

t.interval(0.95, len(police_rates)-1, loc=sample_mean, scale=stats.sem(police_rates, axis=None, ddof=0))  # 95% confidence interval

# with a confidence of 95% we can estimate that the true mean of all police salaries lies between (86177.06785530433, 86795.76115096245)

(86177.06785530433, 86795.76115096245)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here:

